## Import

In [1]:
pip install googletrans

Note: you may need to restart the kernel to use updated packages.


You are using pip version 18.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [25]:
import os
import tweepy as tw
import pandas as pd
import re

import json
import pickle
import time

#translate
from googletrans import Translator
translator = Translator(service_urls=['translate.google.com'])

# Twitter

## Tweepy credentials

In [2]:
consumer_key= 'enter here'
consumer_secret= 'enter here'
access_token= 'enter here'
access_token_secret= 'enter here'
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [3]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

## Search

In [4]:
search_words = "cpi"
date_since = "2019-01-16"

In [6]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="hi",
              since=date_since,tweet_mode="extended").items(2300)
l=[]
# Iterate and print tweets
for tweet in tweets:
    # print(re.sub("\n","",tweet.full_text))
    print("-",end="")
    try:
        l.append(re.sub("\n","",tweet.retweeted_status.full_text))
    except:
        l.append(re.sub("\n","",tweet.full_text))
l=list(set(l))

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
(l)

['हो जाएगा. ताऊ आपको याद होना चाहिए. अमेरिकी कम्युनिष्ट पार्टी (साम्यवादी =काला समाज) trump bhaiya को बहुत पीड़ित किया था 2006 तक पीड़ित किया है. भारत मे CPI=BJP है. बनकर अलग है सिपाही वही है. नीति चले,शीघ्र राशि दिलाया आदित्य भी भारत से जाएगा. आदित्य को मोह खत्म करे.',
 'ब्रेकिंग न्यूज""सुना है आज नया कॉन्टेक्ट दिया है अमितशाह ने  ओवैसी को बंगाल के लिए""TMC, Congres,CPi ,C/2 को हराने के लिए""जितना सीट हराएगा उसके हिसाब से 20 करोड़ देगा भाजपा ओवैसी को""पहचानो मुस्लिम भाई इस दोगले को          ',
 'CPI (M) नेता सुनील चोपड़ा ने बीजेपी पर बोला हमला हल्ला_बोल लाइव    WestBengal  ',
 ' Historical_Day 20_NOV_1991 इटावा से चुनकर मान्यवर कांशीराम साहब संसद में पहुंचे थे, वो भी अपने बलबूते पर भाजपा, कोंग्रेस, समाजवादी पार्टी, जनता दल, CPI/CPM जैसी मनुवादी पार्टियों को रगड़ रगडकर! SaluteManyavarKanshiRamSaheb',
 '                      1000 वोट से कम मार्जिन पर जीतने वाली 11 सीट जिसमे 5 jdu,3 rjd,1ljp,1 cpi,1 ind  ',
 'घबराने की ज़रूरत नहीं है...NDA और मज़बूत होगी .125+1(BSP)+5(AIMIM)+2(CPI)+2(CPI

In [8]:
%%time
for i in range(len(l)):
    l[i] = re.sub("(@[A-Za-z0-9_]+)|(http\S+)|(#)|(&amp;)"," ",l[i])
    l[i] = re.sub("।",".",l[i])
    l[i] = deEmojify(l[i])

Wall time: 10.4 ms


In [13]:
len(l)

180

In [11]:
c=[]
for i in l:
    for j in i.split("."):
        if(j!=""):
            c.append(j)

In [14]:
c

['हो जाएगा',
 ' ताऊ आपको याद होना चाहिए',
 ' अमेरिकी कम्युनिष्ट पार्टी (साम्यवादी =काला समाज) trump bhaiya को बहुत पीड़ित किया था 2006 तक पीड़ित किया है',
 ' भारत मे CPI=BJP है',
 ' बनकर अलग है सिपाही वही है',
 ' नीति चले,शीघ्र राशि दिलाया आदित्य भी भारत से जाएगा',
 ' आदित्य को मोह खत्म करे',
 'ब्रेकिंग न्यूज""सुना है आज नया कॉन्टेक्ट दिया है अमितशाह ने  ओवैसी को बंगाल के लिए""TMC, Congres,CPi ,C/2 को हराने के लिए""जितना सीट हराएगा उसके हिसाब से 20 करोड़ देगा भाजपा ओवैसी को""पहचानो मुस्लिम भाई इस दोगले को          ',
 'CPI (M) नेता सुनील चोपड़ा ने बीजेपी पर बोला हमला हल्ला_बोल लाइव    WestBengal  ',
 ' Historical_Day 20_NOV_1991 इटावा से चुनकर मान्यवर कांशीराम साहब संसद में पहुंचे थे, वो भी अपने बलबूते पर भाजपा, कोंग्रेस, समाजवादी पार्टी, जनता दल, CPI/CPM जैसी मनुवादी पार्टियों को रगड़ रगडकर! SaluteManyavarKanshiRamSaheb',
 '                      1000 वोट से कम मार्जिन पर जीतने वाली 11 सीट जिसमे 5 jdu,3 rjd,1ljp,1 cpi,1 ind  ',
 'घबराने की ज़रूरत नहीं है',
 'NDA और मज़बूत होगी ',
 '125

In [27]:
%%time
final=[]
for q in c:
    try:
        final.append(translator.translate(q,src="hi",dest="en").text)
    except:
        pass

Wall time: 2min 21s


In [28]:
final

['Will be done',
 'Tau you must remember',
 'The American Communist Party (communist = black society) had suffered a lot on trump bhaiya till 2006.',
 'CPI = BJP in India',
 'The soldier is different',
 'Let the policy go, Aditya will get funds from India soon',
 'End adity to aditya',
 'Breaking News "" Heard today has given a new contact. Amit Shah has given Owaisi to Bengal "" to defeat TMC, Congres, CPi, C / 2 "" 20 Crores according to the number of seats he will beat "Identify Owaisi" " Muslim brother to this bastard',
 'CPI (M) leader Sunil Chopra attacks BJP',
 'Historical_Day 20_NOV_1991 Manyavar Kanshi Ram Saheb, elected from Etawah, had reached Parliament, rubbing the Manuwadi parties like BJP, Congress, Samajwadi Party, Janata Dal, CPI / CPM on his own! SaluteManyavarKanshiRamSaheb',
 '11 seats with less than 1000 votes margin, 5 jdu, 3 rjd, 1ljp, 1 cpi, 1 ind',
 'No need to panic',
 'NDA will be stronger',
 '125 + 1 (BSP) +5 (AIMIM) +2 (CPI) +2 (CPIM) +10 (INC) +1 (independ

In [26]:
translator.translate("विधायक अमानतुल्ला खान तीसरी बार चुने गए दिल्ली वक्फ बोर्ड के अध्यक्ष ",src="hi",dest="en").text

'MLA Amanatullah Khan elected for the third time Chairman of Delhi Waqf Board'

In [29]:
with open("cpi_hi.txt", "wb") as fp:   #Pickling
      pickle.dump(final, fp)

In [19]:
with open('congress_en.txt',"rb") as fp:
    b = pickle.load(fp)

In [20]:
b

['Over 700 lectures, 550 speakers, 200 scientific sessions, 67 industry sessions and countless learning opportunities. If you missed any of  eadvvirtual, content is available on-demand until 31st January 2021 &gt;&gt;    eadv_  congress  EADVVirtual  ',
 'గ్రేటర్ లో కాంగ్రెస్ కు దారుణ పరిస్థితి | Worst Situation For Congress | Greater Elections 2020| hmtv Congress   GHMCElections2020   hmtvWatch here &gt;&gt;   ',
 '  Finally, something Sen. Cornyn and I agree on. Congress needs to act and act quickly to pass the next pandemic aid package.  pandemic  BidenTransition  Congress',
 'EU-Turkey separation widening, relations approaching breaking point – Borrell      USA  Congress  Senat    EuropeanUnion  Cyprus      MiddleEast  EastMed',
 ' Goa:  Congress interim president  SoniaGandhi and her son and party leader  RahulGandhi arrive in  Panaji. Doctors had earlier advised Sonia Gandhi to spend time in a less polluted place  ',
 '  Perhaps instead of wasting time and taxpayer money on a wor

## Timeline


In [ ]:
public_tweets = api.home_timeline(tweet_mode="extended")
l=[]
for tweet in public_tweets:
    l.append(re.sub("\n","",tweet.full_text))
l